In [37]:
import pandas as pd
import numpy as np
import joblib
import requests
import pickle


dict = joblib.load('dicts/translations.pkl')

In [40]:
translations = {
    '0191': 'Agriculture, including support for agriculture and post-harvest',
    '0192': 'Livestock, including support for livestock',
    '0280': 'Forestry production; fishing and aquaculture',
    '0580': 'Coal and non-metallic minerals extraction',
    '0680': 'Oil and gas extraction, including support activities',
    '0791': 'Iron ore extraction, including beneficiation and agglomeration',
    '0792': 'Non-ferrous metallic minerals extraction, including beneficiation',
    '1091': 'Meat products, including dairy and fish products',
    '1092': 'Sugar manufacturing and refining',
    '1093': 'Other food products',
    '1100': 'Beverage manufacturing',
    '1200': 'Tobacco products manufacturing',
    '1300': 'Textile products manufacturing',
    '1400': 'Clothing and accessories manufacturing',
    '1500': 'Footwear and leather products manufacturing',
    '1600': 'Wood products manufacturing',
    '1700': 'Pulp, paper and paper products manufacturing',
    '1800': 'Printing and reproduction of recordings',
    '1991': 'Petroleum refining and coke production',
    '1992': 'Biofuels manufacturing',
    '2091': 'Organic and inorganic chemicals, resins, and elastomers manufacturing',
    '2092': 'Pesticides, disinfectants, paints, and various chemicals manufacturing',
    '2093': 'Cleaning products, cosmetics/perfume, and personal hygiene products manufacturing',
    '2100': 'Pharmaceutical and pharmaceutical products manufacturing',
    '2200': 'Rubber and plastic products manufacturing',
    '2300': 'Non-metallic mineral products manufacturing',
    '2491': 'Pig iron/ferroalloys production, steelmaking, and seamless steel pipes',
    '2492': 'Non-ferrous metals metallurgy and metal casting',
    '2500': 'Metal products manufacturing, except machinery and equipment',
    '2600': 'Computing equipment, electronic and optical products manufacturing',
    '2700': 'Electrical equipment manufacturing',
    '2800': 'Mechanical machinery and equipment manufacturing',
    '2991': 'Automobiles, trucks, and buses manufacturing, except parts',
    '2992': 'Parts and accessories manufacturing for motor vehicles',
    '3000': 'Other transportation equipment manufacturing, except motor vehicles',
    '3180': 'Furniture and diverse industries products manufacturing',
    '3300': 'Maintenance, repair, and installation of machinery and equipment',
    '3500': 'Electricity, natural gas, and other utilities',
    '3680': 'Water, sewage, and waste management',
    '4180': 'Construction',
    '4500': 'Motor vehicles and motorcycles trade and repair',
    '4680': 'Wholesale and retail trade, except motor vehicles',
    '4900': 'Land transport',
    '5000': 'Water transport',
    '5100': 'Air transport',
    '5280': 'Storage, transportation support activities, and mail',
    '5500': 'Accommodation',
    '5600': 'Food services',
    '5800': 'Publishing and integrated printing',
    '5980': 'Television, radio, cinema, and sound/image recording/editing activities',
    '6100': 'Telecommunications',
    '6280': 'Systems development and other information services',
    '6480': 'Financial intermediation, insurance, and supplementary pension',
    '6800': 'Real estate activities',
    '6980': 'Legal, accounting, consulting, and company headquarters activities',
    '7180': 'Architectural, engineering, testing/technical analysis, and R&D services',
    '7380': 'Other professional, scientific, and technical activities',
    '7700': 'Non-real estate rentals and management of intellectual property assets',
    '7880': 'Other administrative activities and complementary services',
    '8000': 'Surveillance, security, and investigation activities',
    '8400': 'Public administration, defense, and social security',
    '8591': 'Public education',
    '8592': 'Private education',
    '8691': 'Public health',
    '8692': 'Private health',
    '9080': 'Artistic, creative, and entertainment activities',
    '9480': 'Associative organizations and other personal services',
    '9700': 'Domestic services',
    '9999': 'Total product',
    '84002': 'Social security and social assistance services',
    '85911': 'Public education',
    '85921': 'Private education',
    '86911': 'Public health',
    '86921': 'Private health',
    '90801': 'Arts, culture, sports, and recreation services',
    '94801': 'Employer organizations, unions, and other associative services',
    '94802': 'Maintenance of computers, phones, and household items',
    '94803': 'Personal services',
    '97001': 'Domestic services'
}

joblib.dump(translations, 'dicts/translations.pkl')

['dicts/translations.pkl']

In [26]:
import pandas as pd
import numpy as np
import requests
import pickle

class MatrixLoader:
    def __init__(self, year, agg):
        self.base_url = 'https://github.com/GuilhermeZiegler/input_output/raw/master/input_output/data/iom'
        self.year = year
        self.agg = agg
        self.file_path = f'{self.base_url}/{self.agg}/{self.year}/MIP-BR-CN10-68S-{self.year}.xlsx'
        self.matrix = {}
        self.translations = self.load_translations()
        self.load_data()

    def load_translations(self):
        url = "https://github.com/GuilhermeZiegler/input_output/raw/master/input_output/dicts/translations.pkl"
        response = requests.get(url)
        response.raise_for_status()
        return pickle.loads(response.content)

    def load_data(self):
        sheet_name_map = {
            'Producao': 'P',
            'Usos PxS': 'PxS',
            'Usos SxS': 'SxS',
            'Mat A Coef Tec': 'A',
            'Inv Leontief': 'Leontief_Inv',
            'Importacoes': 'M',
            'Imposto Import': 'T',
            'ICMS': 'ICMS',
            'IPI': 'IPI',
            'OIIL': 'OIIL',
            'MG Com': 'MG_Com',
            'MG Transp': 'MG_Transp'
        }

        xls = pd.ExcelFile(self.file_path)
        
        for sheet_name in xls.sheet_names:
            if sheet_name == 'Referência':
                continue
            
            translated_name = sheet_name_map.get(sheet_name, sheet_name)
            self.extract_indices_and_matrix(translated_name, sheet_name, xls)

    def extract_indices_and_matrix(self, translated_name, sheet_name, xls):
        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
        row_start_index = 4
        row_indexes = df.iloc[row_start_index:, :3].reset_index(drop=True)
        col_start_row, col_end_row = 1, 3
        col_start_col = 3
        column_indexes = df.iloc[col_start_row:col_end_row+1, col_start_col:].reset_index(drop=True)
        data_matrix = df.iloc[row_start_index:, col_start_col:].reset_index(drop=True)
        data_matrix = data_matrix.dropna(axis=1, how='all')
        
        row_indexes_translated = row_indexes.applymap(lambda x: self.translations.get(str(x).strip(), x))
        display(row_indexes_translated )
        column_indexes_translated = column_indexes.applymap(lambda x: self.translations.get(str(x).strip(), x))
        
        self.matrix[translated_name] = {
            'row_indexes': row_indexes_translated,
            'column_indexes': column_indexes_translated,
            'data_matrix': data_matrix,
            'aligned_matrix': self.align_matrix(row_indexes_translated, column_indexes_translated, data_matrix)
        }

    def align_matrix(self, row_indexes, column_indexes, data_matrix):
        col_len_data = data_matrix.shape[1]
        col_len_index = column_indexes.shape[1]

        if col_len_data < col_len_index:
            for _ in range(col_len_index - col_len_data):
                data_matrix[f'NaN_{_}'] = np.nan
        elif col_len_data > col_len_index:
            data_matrix = data_matrix.iloc[:, :col_len_index]
        if row_indexes.shape[1] > 2:
            row_indexes = row_indexes.iloc[:, :2]
        elif row_indexes.shape[1] < 2:
            row_indexes['dummy'] = np.nan
        if column_indexes.shape[0] > 2:
            column_indexes = column_indexes.iloc[:2, :]
        elif column_indexes.shape[0] < 2:
            extra_row = pd.Series([np.nan] * column_indexes.shape[1])
            column_indexes = column_indexes.append(extra_row, ignore_index=True)
            
        row_index = pd.MultiIndex.from_frame(row_indexes)
        row_index.names = ['codes', 'sectors']
        col_index = pd.MultiIndex.from_frame(column_indexes.T)
        col_index.names = ['codes', 'sectors']
        aligned_df = pd.DataFrame(data_matrix.values, index=row_index, columns=col_index)
        aligned_df = aligned_df.reindex(index=row_index, columns=col_index)

        return aligned_df

    def get_aligned_matrix(self, sheet_name):
        return self.matrix.get(sheet_name, {}).get('aligned_matrix', None)

    def display_aligned_matrices(self):
        for sheet_name, data in self.matrix.items():
            print(f"Sheet: {sheet_name}")
            print("Aligned Matrix:")
            display(data['aligned_matrix'])
            print("\n")

# Example usage
# loader = MatrixLoader(2024, 'some_agg_value')
# loader.display_aligned_matrices()




In [27]:
# Exemplo de uso
year = '2018'
agg = '68S'
min_68S_2018 = MatrixLoader(year, agg)



,0,1,2
0,0191,"Agricultura, inclusive o apoio à agricultura e...",1.0
1,0192,"Pecuária, inclusive o apoio à pecuária",2.0
2,0280,Produção florestal; pesca e aquicultura,3.0
3,0580,Extração de carvão mineral e de minerais não-m...,4.0
4,0680,"Extração de petróleo e gás, inclusive as ativi...",5.0
...,...,...,...
66,9480,Organizações associativas e outros serviços pe...,67.0
67,9700,Domestic_Services,68.0
68,NaN,Total\ndo produto,69.0
69,NaN,NaN,NaN


,0,1,2
0,01911,"Arroz, trigo e outros cereais",1.0
1,01912,Milho em grão,2.0
2,01913,"Algodão herbáceo, outras fibras da lav. tempor...",3.0
3,01914,Cana-de-açúcar,4.0
4,01915,Soja em grão,5.0
...,...,...,...
157,NaN,NaN,NaN
158,NaN,NaN,NaN
159,NaN,NaN,NaN
160,NaN,NaN,NaN


,0,1,2
0,01911,"Agricultura, inclusive o apoio à agricultura e...",1.0
1,01912,"Pecuária, inclusive o apoio à pecuária",2.0
2,01913,Produção florestal; pesca e aquicultura,3.0
3,01914,Extração de carvão mineral e de minerais não-m...,4.0
4,01915,"Extração de petróleo e gás, inclusive as ativi...",5.0
...,...,...,...
97,NaN,NaN,NaN
98,NaN,NaN,NaN
99,NaN,NaN,NaN
100,NaN,NaN,NaN


,0,1,2
0,01911,"Agricultura, inclusive o apoio à agricultura e...",1.0
1,01912,"Pecuária, inclusive o apoio à pecuária",2.0
2,01913,Produção florestal; pesca e aquicultura,3.0
3,01914,Extração de carvão mineral e de minerais não-m...,4.0
4,01915,"Extração de petróleo e gás, inclusive as ativi...",5.0
...,...,...,...
64,23002,Saúde privada,65.0
65,23003,"Atividades artísticas, criativas e de espetáculos",66.0
66,24911,Organizações associativas e outros serviços pe...,67.0
67,97001,Domestic_Services,68.0


,0,1,2
0,01911,"Agricultura, inclusive o apoio à agricultura e...",1.0
1,01912,"Pecuária, inclusive o apoio à pecuária",2.0
2,01913,Produção florestal; pesca e aquicultura,3.0
3,01914,Extração de carvão mineral e de minerais não-m...,4.0
4,01915,"Extração de petróleo e gás, inclusive as ativi...",5.0
...,...,...,...
64,23002,Saúde privada,65.0
65,23003,"Atividades artísticas, criativas e de espetáculos",66.0
66,24911,Organizações associativas e outros serviços pe...,67.0
67,97001,Domestic_Services,68.0


...

,0,1,2
0,01911,"Arroz, trigo e outros cereais",1.0
1,01912,Milho em grão,2.0
2,01913,"Algodão herbáceo, outras fibras da lav. tempor...",3.0
3,01914,Cana-de-açúcar,4.0
4,01915,Soja em grão,5.0
...,...,...,...
124,94801,"Organizações patronais, sindicais e outros ser...",125.0
125,94802,"Manutenção de computadores, telefones e objeto...",126.0
126,94803,Serviços pessoais,127.0
127,97001,Domestic_Services,128.0


,0,1,2
0,01911,"Arroz, trigo e outros cereais",1.0
1,01912,Milho em grão,2.0
2,01913,"Algodão herbáceo, outras fibras da lav. tempor...",3.0
3,01914,Cana-de-açúcar,4.0
4,01915,Soja em grão,5.0
...,...,...,...
124,94801,"Organizações patronais, sindicais e outros ser...",125.0
125,94802,"Manutenção de computadores, telefones e objeto...",126.0
126,94803,Serviços pessoais,127.0
127,97001,Domestic_Services,128.0


,0,1,2
0,01911,"Arroz, trigo e outros cereais",1.0
1,01912,Milho em grão,2.0
2,01913,"Algodão herbáceo, outras fibras da lav. tempor...",3.0
3,01914,Cana-de-açúcar,4.0
4,01915,Soja em grão,5.0
...,...,...,...
124,94801,"Organizações patronais, sindicais e outros ser...",125.0
125,94802,"Manutenção de computadores, telefones e objeto...",126.0
126,94803,Serviços pessoais,127.0
127,97001,Domestic_Services,128.0


,0,1,2
0,01911,"Arroz, trigo e outros cereais",1.0
1,01912,Milho em grão,2.0
2,01913,"Algodão herbáceo, outras fibras da lav. tempor...",3.0
3,01914,Cana-de-açúcar,4.0
4,01915,Soja em grão,5.0
...,...,...,...
124,94801,"Organizações patronais, sindicais e outros ser...",125.0
125,94802,"Manutenção de computadores, telefones e objeto...",126.0
126,94803,Serviços pessoais,127.0
127,97001,Domestic_Services,128.0


,0,1,2
0,01911,"Arroz, trigo e outros cereais",1.0
1,01912,Milho em grão,2.0
2,01913,"Algodão herbáceo, outras fibras da lav. tempor...",3.0
3,01914,Cana-de-açúcar,4.0
4,01915,Soja em grão,5.0
...,...,...,...
124,94801,"Organizações patronais, sindicais e outros ser...",125.0
125,94802,"Manutenção de computadores, telefones e objeto...",126.0
126,94803,Serviços pessoais,127.0
127,97001,Domestic_Services,128.0


In [28]:
for key, item in min_68S_2018.matrix.items():
    print(key)
   


P
PxS
SxS
A
Leontief_Inv
M
T
ICMS
IPI
OIIL
MG_Com
MG_Transp


In [29]:
min_68S_2018.matrix['P']

{'row_indexes':        0                                                  1     2
 0   0191  Agricultura, inclusive o apoio à agricultura e...   1.0
 1   0192             Pecuária, inclusive o apoio à pecuária   2.0
 2   0280            Produção florestal; pesca e aquicultura   3.0
 3   0580  Extração de carvão mineral e de minerais não-m...   4.0
 4   0680  Extração de petróleo e gás, inclusive as ativi...   5.0
 ..   ...                                                ...   ...
 66  9480  Organizações associativas e outros serviços pe...  67.0
 67  9700                                  Domestic_Services  68.0
 68   NaN                                  Total\ndo produto  69.0
 69   NaN                                                NaN   NaN
 70   NaN                                        Conferência   NaN
 
 [71 rows x 3 columns],
 'column_indexes':                              3              4    \
 0                          01911          01912   
 1  Arroz, trigo e outros cereais

In [30]:
min_68S_2018.translations

{'Agricultura, silvicultura, exploração florestal': 'Agriculture_Forestry_Logging',
 'Pecuária e pesca': 'Livestock_and_Fishing',
 'Petróleo e gás natural': 'Oil_and_Natural_Gas',
 'Minério de ferro': 'Iron_Ore',
 'Outros da indústria extrativa': 'Other_Extractive_Industries',
 'Alimentos e Bebidas': 'Food_and_Beverages',
 'Produtos do fumo': 'Tobacco_Products',
 'Têxteis': 'Textiles',
 'Artigos do vestuário e acessórios': 'Apparel_and_Accessories',
 'Artefatos de couro e calçados': 'Leather_Goods_and_Footwear',
 'Produtos de madeira - exclusive móveis': 'Wood_Products_Excluding_Furniture',
 'Celulose e produtos de papel': 'Pulp_and_Paper_Products',
 'Jornais, revistas, discos': 'Newspapers_Magazines_Records',
 'Refino de petróleo e coque': 'Petroleum_Refining_and_Coke',
 'Álcool': 'Alcohol',
 'Produtos químicos': 'Chemical_Products',
 'Fabricação de resina e elastômeros': 'Resin_and_Elastomer_Manufacturing',
 'Produtos farmacêuticos': 'Pharmaceutical_Products',
 'Defensivos agrícolas'